# Cognitive, behavioral and social data
**Author**: Mattia Brocco

MERGE OF DATASETS FOR **R_NEO_PI**
```python
a = pd.read_excel(data_dir + "\\R_NEO_PI_Faked.xlsx")
b = pd.read_excel(data_dir + "\\R_NEO_PI_Honest.xlsx")

a.columns = [" ".join([pd.Series(a.columns).apply(lambda s: np.nan if "Unnamed"
                                                  in s else s).fillna(method = "ffill").tolist()[i],
                       a.loc[0][i]]) for i in range(len(a.columns))]
b.columns = [" ".join([pd.Series(b.columns).apply(lambda s: np.nan if "Unnamed"
                                                  in s else s).fillna(method = "ffill").tolist()[i],
                       b.loc[0][i]]) for i in range(len(b.columns))]

a = a.drop(0).reset_index(drop = True)
b = b.drop(0).reset_index(drop = True)

a["CONDITION"] = "FAKE"
b["CONDITION"] = "HONEST"

pd.concat([a, b], ignore_index = True).to_excel(data_dir + "\\R_NEO_PI.xlsx", index = False)
```

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import support
import evaluation
from engine import Classification

%load_ext autoreload
%autoreload 2

data_dir = ".\\data"

pd.options.display.max_columns = 500

### Datasets at hand

In [2]:
data_descr = pd.DataFrame(dict(zip([f for f in os.listdir(data_dir) if "parquet" in f],
                                   [pd.read_parquet(f"{data_dir}\\{f}").shape for f in os.listdir(data_dir)
                                    if "parquet" in f]))).T.reset_index()
data_descr = data_descr.rename(columns = {"index": "Data", 0: "Sample size",
                                          1: "Features"})
data_descr

,Data,Sample size,Features
0,BF_df_CTU.parquet,442,11
1,BF_df_OU.parquet,460,11
2,BF_df_V.parquet,486,11
3,DT_df_CC.parquet,482,28
4,DT_df_JI.parquet,864,28
5,IADQ_df.parquet,450,10
6,IESR_df.parquet,358,23
7,NAQ_R_df.parquet,712,23
8,PCL5_df.parquet,402,21
9,PHQ9_GAD7_df.parquet,1118,17


## Design a pipeline
***

The goal is to find a stable subset of features across datasets that performs roughly the same across different classifiers. Accoringly, we can define a "good" feature selection procedure, one that does not depend on a specific model, but that allows models to perform roughly the same, and for all the datasets within the scope.

##### DESCRIPTION
1. A given dataset is split in training and test. For every feature, the mean and the standard deviation are computed in order to scale that feauture: $Z=\frac{X-\mu}{\sigma}$. Scaling on the test set is carried out using the same values computed for training data.
2. The actual phase of the selection of a subset of features occurs through a 3-step process:  
    * Train a Decision Tree and apply minimal cost-complexity pruning. <sup>[1]</sup>  
    * Train a Random Forest that expolits gradient boosting <sup>[2]</sup>, in which each tree retains the cost-complexity parameter obtained in the previous step.  
    * Compute permutation importance on this Random Forest <sup>[3]</sup>.  
    * Perform a one-sample test on the mean, given the distribution obtained for each feature importance (with confidence level at 99.999%). This way only features whose importance is significantly greater than zero are retained, all the others are discarded. We call this subset of feature $A^*$  
    * Perform a Wilks test <sup>[4]</sup> comparing two logistic regressions, one fitted with the full set of features, the other with $A^*$. By accepting the null hypothesis of the test (at 95% confidence level), the assertion "the ration  between the likelihoods of the two (nested) models is one".  
    * Train an arbitrary amount of different models in order to assess the quality of the feature selection procedure. If all models show very close accuracy, then the procedure proves to provide a model-indifferent subset $A^*$ of features.


##### SOURCES
* [How can I get statistics to compare nested models in a logistic regression in SPSS?](https://www.ibm.com/support/pages/how-can-i-get-statistics-compare-nested-models-logistic-regression-spss)
* [Likelihood-ratio test](https://en.wikipedia.org/wiki/Likelihood-ratio_test)

##### SUPPORTING PAPERS
[1] L. Breiman, J. Friedman, R. Olshen, and C. Stone. *Classification and Regression Trees*. Wadsworth, Belmont, CA, 1984.  
[2] J. Friedman, Greedy Function Approximation: *A Gradient Boosting Machine*, The Annals of Statistics, Vol. 29, No. 5, 2001  
[3] Leo Breiman. Random forests. *Machine learning*, 45(1):5-32, 2001.  
[4] Li, Bing; Babu, G. Jogesh (2019). *A Graduate Course on Statistical Inference*. Springer. p. 331

In [ ]:
### PIPELINE
# Organize datasets
data_collection = {}
for dataset in [f for f in os.listdir(data_dir) if "parquet" in f]:
    print(dataset.split(".")[0])
    X_tr, X_ts, y_tr, y_ts = Classification().prepare_data(f"{data_dir}\\{dataset}", "CONDITION")
    var_sel = Classification().variable_selection(X_tr, X_ts, y_tr, y_ts)
    perfs, stability = Classification().benchmark_models(X_tr, X_ts, y_tr, y_ts, var_sel)
    data_collection[dataset.split(".")[0]] = [X_tr, X_ts, y_tr, y_ts, var_sel, perfs, stability]
    print(var_sel)
    print(perfs)
    print()
    print("==> STABILITY SCORE: {}".format(stability))
    print("-" * 100)
    print("-" * 100)
    print("-" * 100)
    print()

BF_df_CTU


  0%|          | 0/100 [00:00<?, ?it/s]

5/5 [==============================] - 0s 2ms/step
Train size: 309
Selected 4 features out of 10
{'Features': [2, 4, 6, 7], 'Wilks test p-value': 0.9893064, 'High correlation': False}
                     Accuracy  FI 1  FI 2  FI 3  FI 4
Full Logit           0.774436   NaN   NaN   NaN   NaN
Logistic Regression  0.812030   6.0   2.0   4.0   7.0
SVC                  0.842105   6.0   2.0   4.0   7.0
Random Forest        0.827068   6.0   7.0   2.0   4.0
Neural Network       0.812030   4.0   7.0   6.0   2.0

==> STABILITY SCORE: 0.5
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------

BF_df_OU


  0%|          | 0/100 [00:00<?, ?it/s]

5/5 [==============================] - 0s 2ms/step
Train size: 322
Selected 3 features out of 10
{'Features': [4, 6, 7], 'Wilks test p-value': 1.0, 'High correlation': False}
                     Accuracy  FI 1  FI 2  FI 3
Full Logit           0.833333   NaN   NaN   NaN
Logistic Regression  0.811594   6.0   4.0   7.0
SVC                  0.818841   6.0   4.0   7.0
Random Forest        0.782609   6.0   4.0   7.0
Neural Network       0.818841   6.0   4.0   7.0

==> STABILITY SCORE: 1.0
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------

BF_df_V


...

7/7 [==============================] - 0s 2ms/step
Train size: 474
Selected 4 features out of 18
{'Features': [6, 7, 8, 11], 'Wilks test p-value': 1.0, 'High correlation': False}
                     Accuracy  FI 1  FI 2  FI 3  FI 4
Full Logit           0.936275   NaN   NaN   NaN   NaN
Logistic Regression  0.892157  11.0   6.0   7.0   8.0
SVC                  0.882353   8.0  11.0   6.0   7.0
Random Forest        0.916667   6.0  11.0   7.0   8.0
Neural Network       0.901961  11.0   8.0   7.0   6.0

==> STABILITY SCORE: 0.417
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------

PRMQ_df


  0%|          | 0/100 [00:00<?, ?it/s]

14/14 [==============================] - 0s 2ms/step
Train size: 982
Selected 5 features out of 16
{'Features': [1, 4, 7, 12, 14], 'Wilks test p-value': 1.0, 'High correlation': False}
                     Accuracy  FI 1  FI 2  FI 3  FI 4  FI 5
Full Logit           0.907583   NaN   NaN   NaN   NaN   NaN
Logistic Regression  0.890995   1.0  12.0   7.0   4.0  14.0
SVC                  0.912322   1.0   7.0  12.0   4.0  14.0
Random Forest        0.888626   1.0   7.0  12.0   4.0  14.0
Neural Network       0.893365   1.0   7.0  12.0   4.0  14.0

==> STABILITY SCORE: 0.867
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------

RAW_DDDT


  0%|          | 0/100 [00:00<?, ?it/s]

10/10 [==============================] - 0s 2ms/step
Train size: 690
Selected 4 features out of 12
{'Features': [0, 1, 9, 10], 'Wilks test p-value': 1.0, 'High correlation': False}
                     Accuracy  FI 1  FI 2  FI 3  FI 4
Full Logit           0.746622   NaN   NaN   NaN   NaN
Logistic Regression  0.746622   1.0  10.0   9.0   0.0
SVC                  0.766892  10.0   1.0   0.0   9.0
Random Forest        0.760135  10.0   1.0   0.0   9.0
Neural Network       0.750000   1.0  10.0   0.0   9.0

==> STABILITY SCORE: 0.667
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------

R_NEO_PI


## Assess quality of analysis
***

In [ ]:
# Compare with correlation matrices
# The greener the higher the correlation
f0, ax0 = plt.subplots(4, 4, figsize = (20, 24))
for n, k in enumerate(data_collection.keys()):
    
    sns.heatmap(pd.DataFrame(data_collection[k][0]).corr().abs(),
                vmin = 0, vmax = 1, cmap = "YlGn",
                cbar = False, ax = ax0[n//4, n%4])
    ax0[n//4, n%4].set_title("{} - {} selected"\
                             .format(k, len(data_collection[k][-3]["Features"])))
plt.show()

In [ ]:
# Correlation matrices of only selected features
# The greener the higher the correlation
f1, ax1 = plt.subplots(4, 4, figsize = (20, 24))
for n, k in enumerate(data_collection.keys()):
    
    feats = data_collection[k][-2]["Features"]
    
    annotation = True if len(feats) < 7 else False
    
    sns.heatmap(pd.DataFrame(data_collection[k][0][:,feats]).corr().abs(),
                vmin = 0, vmax = 1, cmap = "YlGn", annot = annotation,
                cbar = False, ax = ax1[n//4, n%4])

    ax1[n//4, n%4].set_title(k)
    
plt.show()

In [ ]:
benchmark = evaluation.lasso_benchmark(data_collection)

benchmark_df = pd.DataFrame(benchmark,
                            columns = ["Dataset", "Algo", "λ min", "λ 1se",
                                       "Zero coeffs at λ min",
                                       "Delta accuracy"]).drop("Algo", axis = 1)
benchmark_df

In [ ]:
#evaluation.graphical_lasso_benchmark(benchmark[7])

## Summarize results
***

In [ ]:
summary = []
for k, v in data_collection.items():
    
    # Dataset name
    # Sample size
    # Training size
    # Initial number of features
    # Selected features
    # Stability
    # ACCURACY: Full Logit
    # ACCURACY: Logistic Regression
    # ACCURACY: SVC
    # ACCURACY: Random Forest
    # ACCURACY: Neural Network
    # Average accuracy (full logit excluded)
    # Accuracy Standard deviation (full logit excluded)
    # Decreased accuracy (mean) w.r.t. full logit
    
    summary += [[k, len(v[2]) + len(v[3]), len(v[2]),
                 v[0].shape[1], len(v[4]["Features"]),
                 v[6], *v[5].tolist(), v[5][1:].mean(),
                 v[5][1:].std(), v[5][1:].mean() - v[5].tolist()[0]]]
    
summary = pd.DataFrame(summary,
                       columns = ["Dataset name", "Sample size", "Training size", "Number of Features",
                                  "Selected Features", "Feat. Stability", "Accuracy - Logit all features",
                                  "Accuracy - Logit", "Accuracy - SVM", "Accuracy - RF",
                                  "Accuracy - MLP", "Avg Acc. on selected features",
                                  "Accuracy Std on selected features", "Acc. diff. wrt Full logit"])

summary = summary.applymap(lambda v: v if isinstance(v, str) else round(v, 3))

summary.sort_values("Accuracy Std on selected features", ascending = False)

In [ ]:
plt.figure(figsize = (20, 7))
plt.bar(x = summary.sort_values("Avg Acc. on selected features",
                                ascending = False)["Dataset name"],
        height = summary.sort_values("Avg Acc. on selected features",
                                     ascending = False)["Avg Acc. on selected features"],
        yerr = summary.sort_values("Avg Acc. on selected features",
                                   ascending = False)["Accuracy Std on selected features"], alpha = .7)
plt.xticks(rotation = 35, fontsize = 13)
plt.yticks(fontsize = 12)
plt.ylabel("Accuracy\n(mean ± std)", fontsize = 13, rotation = 0, labelpad = 50)
plt.title("Accuracy on different datasets", fontsize = 18)
plt.show()

In [ ]:
plt.figure(figsize = (18, 6))
plt.bar(x = summary.sort_values("Accuracy Std on selected features",
                                ascending = False)["Dataset name"],
        height = summary.sort_values("Accuracy Std on selected features",
                                     ascending = False)["Accuracy Std on selected features"], alpha = .7)
plt.xticks(rotation = 35, fontsize = 13)
plt.yticks(fontsize = 12)
plt.ylabel("Stand. Dev.", fontsize = 13, rotation = 0, labelpad = 30)
plt.title("Standard Deviation of Accuracies on different datasets", fontsize = 18)
plt.show()

---
---
---